In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [2]:
! pip install -qU "langchain[openai]"


In [3]:
!pip install langchain-openai langchain-community langgraph \
            docling langchain-docling \
            lancedb tantivy \
            langchain-huggingface


In [4]:
!pip install -qU "langchain[openai]"


In [5]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

Enter API key for OpenAI:  ········


In [6]:
pip install -qU langchain-huggingface


Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install -qU langchain-huggingface


In [2]:
pip install -qU langchain-huggingface


Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install langchain-openai langchain-community langgraph \
            docling langchain-docling \
            lancedb tantivy \
            langchain-huggingface


In [2]:
pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
from docling.chunking import HybridChunker
from langchain_docling import DoclingLoader
from sentence_transformers import SentenceTransformer

# STEP 1: Load and chunk the document using Docling
loader = DoclingLoader(
    file_path="/Users/jamesfoote/Documents/Coding/coding_journal.docx",
    export_type="doc_chunks",
    chunker=HybridChunker(
        tokenizer="sentence-transformers/all-MiniLM-L6-v2",  # works fine for semantic splits
        max_tokens=512
    )
)
docs = loader.load()

# STEP 2: Truncate chunks safely to 512 tokens for the local embedding model
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-base")

def truncate_chunks(docs, max_tokens=512):
    truncated = []
    for doc in docs:
        tokens = tokenizer.encode(doc.page_content, truncation=True, max_length=max_tokens)
        decoded = tokenizer.decode(tokens, skip_special_tokens=True)
        doc.page_content = decoded
        truncated.append(doc)
    return truncated

final_chunks = truncate_chunks(docs)

# STEP 3: Load local embedding model
model = SentenceTransformer("intfloat/e5-base")

# STEP 4: Embed locally (E5 model requires "passage: " prefix)
texts = ["passage: " + doc.page_content for doc in final_chunks]

embeddings = model.encode(
    texts,
    batch_size=8,
    show_progress_bar=True
)

Token indices sequence length is longer than the specified maximum sequence length for this model (608 > 512). Running this sequence through the model will result in indexing errors


tokenizer_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [5]:
pip install lancedb --upgrade

Note: you may need to restart the kernel to use updated packages.


In [6]:
import lancedb

db = lancedb.connect(
    uri="db://learning-project-dwwvuw",
    api_key="sk_47L5XCLGUZCEDG66HC6H5XM55KBXTFKGCHTUEXEMQOCJKNH6ESXA====",  # Replace this with your real key
    region="us-east-1"               # or the region assigned to your project
)

In [7]:
import pandas as pd

# Prepare your data
data = [
    {"text": doc.page_content, "vector": embedding}
    for doc, embedding in zip(final_chunks, embeddings)
]

df = pd.DataFrame(data)

# Create or overwrite the table in your cloud DB
table = db.create_table("journal_embeddings", data=df, mode="overwrite")

In [10]:
# Embed your query using the same local model
query = "Explain what a Python list comprehension does"
query_vector = model.encode(["passage: " + query])[0]

# Search the remote LanceDB cloud table
results = table.search(query_vector).limit(3).to_pandas()
# Check available columns first
print("Available columns:", results.columns)

# Just print text results
for i, row in results.iterrows():
    print(f"\n--- Result {i+1} ---")
    print(row["text"])  # Adjust this key if the column has a different name

print(results.columns)
print(results.head())

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Available columns: Index(['text', 'vector', '_distance'], dtype='object')

--- Result 1 ---
100 days of code : python day 5 – password generator - prompts the user for how many of each character type to include. - input ( ) returns a string - int ( ) converts it to a number so we can use it in loops password _ list = [ ] - starts with an empty list. we ’ ll add characters here.

--- Result 2 ---
100 days of code : python day 5 – password generator convert list to string password = " " for char in password _ list : password + = char - loops through the shuffled list and * * joins each character * * into a string. print ( f " your password is : { password } " ) displays the final password.

--- Result 3 ---
100 days of code : python day 5 – password generator key concepts used in day 5 range ( ), description = creates a sequence of numbers for looping. list. append ( ), description = adds items to the password list. random. choice ( ), description = randomly selects one item from a list.

In [11]:
# Run this code to update my Code Journal

from docling.chunking import HybridChunker
from langchain_docling import DoclingLoader
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import lancedb
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()  # load your .env file with the LANCEDB_API_KEY

# Step 1: Load updated .docx and re-chunk
loader = DoclingLoader(
    file_path="/Users/jamesfoote/Documents/Coding/coding_journal.docx",
    export_type="doc_chunks",
    chunker=HybridChunker(
        tokenizer="sentence-transformers/all-MiniLM-L6-v2",
        max_tokens=512
    )
)
docs = loader.load()

# Step 2: Truncate to token limit
tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-base")
def truncate_chunks(docs, max_tokens=512):
    truncated = []
    for doc in docs:
        tokens = tokenizer.encode(doc.page_content, truncation=True, max_length=max_tokens)
        decoded = tokenizer.decode(tokens, skip_special_tokens=True)
        doc.page_content = decoded
        truncated.append(doc)
    return truncated

final_chunks = truncate_chunks(docs)

# Step 3: Embed locally
model = SentenceTransformer("intfloat/e5-base")
texts = ["passage: " + doc.page_content for doc in final_chunks]
embeddings = model.encode(texts, batch_size=8, show_progress_bar=True)

# Step 4: Push to LanceDB (overwrite mode)
db = lancedb.connect(
    uri="db://learning-project-dwwvuw",
    api_key=os.getenv("LANCEDB_API_KEY"),
    region="us-east-1"
)

data = [
    {"text": doc.page_content, "vector": vector}
    for doc, vector in zip(final_chunks, embeddings)
]

df = pd.DataFrame(data)
table = db.create_table("journal_embeddings", data=df, mode="overwrite")

print("✅ Journal successfully updated in LanceDB.")

Token indices sequence length is longer than the specified maximum sequence length for this model (608 > 512). Running this sequence through the model will result in indexing errors


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


ValueError: api_key is required to connect to LanceDB cloud: db://learning-project-dwwvuw